In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score # 정밀도를 계산하기 위해 import 한다.
from sklearn.metrics import recall_score # 재현율을 계산하기 위해 import 한다.
from sklearn.metrics import f1_score # f1 score를 계산하기 위해 import 한다.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

머신러닝의 학습 방법은 크게 지도 학습(Supervised Learning)과 비지도 학습(Unsupervised Learning)으로 나눌 수 있다.  
지도 학습이란 라벨링이된 데이터를 학습시키는 것을 의미하며, 비지도 학습은 라벨링이 되지 않은 데이터를 학습시키는 것이다. 이때, 라벨링이란 학습 데이터에 정답이 표시된 것을 의미하며, 정답 부분을 레이블(타겟, 클래스, 실제값) 데이터라고 한다.

레이블의 형태에 따라 지도 학습은 세부적으로 두 가지 종류로 나눌 수 있다. 레이블이 범주형인 경우에는 분류(Classification) 문제라고 하고, 연속형 숫자인 경우에는 회귀(Regression) 문제라고 한다. 레이블 데이터의 형태를 파악하고 풀려는 문제가 분류 문제인지 회귀 문제인지 파악하는 것이 중요하다.

k-최근접 이웃(k-Nearest Neighbor, kNN) 알고리즘

k-최근접 이웃 알고리즘은 이해하기 쉽고, 자주 사용되는 방법으로 비교 대상이 되는 데이터 포인트 주변에 가장 가까이 존재하는 k개의 데이터와 비교해 가장 가까운 데이터 종류로 판별한다.

k-최근접 이웃 알고리즘은 학습 과정에서 게으른 학습(Lazy Learning) 방법을 사용한다.  
게으른 학습은 학습 데이터 전체를 메모리상에 보관하면서 테스트 데이터가 새로 들어왔을 때 바로 학습하는 것을 의미한다. 학습 데이터를 메모리에 보관하므로 추가적인 학습 시간없이 곧바로 학습 결과를 얻을 수 있다는 장점이 있지만 데이터가 지나치게 커서 메모리에 보관할 수 없을 경우 사용할 수 없다는 단점이 있다.

게으른 학습의 반대말은 열정적 학습(Eager Learning)이라 하는데, 학습 데이터로 일정 기간 학습시킨 후 학습시킨 모델을 기반으로 테스트에 적용하는 방법으로 학습 데이터는 학습시에만 메모리에 보관되며 학습 이후에 테스트 데이터를 분류, 예측할 때 메모리에 보관할 필요가 없다.

게으른 학습과 열정적 학습의 차이는 학습 시간의 필요 유무에 따라 나뉘고 열정적 학습은 게으른 학습보다 메모리를 효율적으로 사용할 수 있다는 장점이 있지만 학습 시간이 오래 걸린다는 단점이 있다.

붓꽃 데이터를 사용해 붓꽃 종류를 분류하는 모델을 생성하고 학습시킨다.

In [2]:
# 데이터 불러오기
raw_data = datasets.load_iris() # 사이킷런이 제공하는 붓꽃 데이터를 불러온다.
# from sklearn.datasets import load_iris
# from sklearn.datasets import *
# raw_data = load_iris()

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)

# 학습 데이터와 테스트 데이터로 분할
# train_test_split() 메소드로 피쳐, 레이블 데이터를 넘겨서 학습 데이터와 테스트 데이터로 나눈다.
# train_size 속성으로 학습 데이터로 사용할 데이터의 비율을 지정한다.
# test_size 속성으로 테스트 데이터로 사용할 데이터의 비율을 지정한다.
# train_size, test_size 속성을 생략하면 학습 데이터와 테스트 데이터를 75 : 25 비율로 분할한다.
# random_sate 속성을 지정하면 실행할 때 마다 매번 같은 데이터를 얻어올 수 있다. => 항상 같은 결과가 나온다.
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 스케일러는 학습 데이터를 기반으로 실행해야 하므로 학습 데이터는 표준화 후 적용하고 테스트 데이터는
# 학습 데이터에 표준화되 스케일러에 적용만 시킨다.
# std_scale.fit(x_train) # 학습 데이터를 스케일러로 표준화 한다.
# x_train = std_scale.transform(x_train) # 표준화된 스케일러에 학습 데이터를 적용한다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화 하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화 스케일러에 적용한다.

# 모델 생성 후 데이터 학습
from sklearn.neighbors import KNeighborsClassifier # k-최근접 이웃 알고리즘을 사용하기 위해 import 한다.
# n_neighbors 속성으로 이웃의 개수를 지정해서 k-최근접 이웃 모델을 만든다.
clf = KNeighborsClassifier(n_neighbors=2)
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 k-최근접 이웃 모델을 학습시킨다.
clf.fit(x_train, y_train)

(150, 4) (150,)
(112, 4) (38, 4) (112,) (38,)


KNeighborsClassifier(n_neighbors=2)

학습된 모델로 테스트 데이터를 예측한다.

In [3]:
# predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


학습된 모델을 평가한다.

In [4]:
# 정확도 평가
# accuracy_score() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 정확도를 계산한다.
accuracy = accuracy_score(y_test, predict)
print(accuracy)

0.9473684210526315


In [5]:
# 정밀도 평가
# precision_score() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 정밀도를 계산한다.
# average 속성으로 정밀도 계산 방법을 지정한다. 생략시 기본값은 'binary'이고 클래스가 딱 2개일 때 사용한다.
# 클래스가 3개 이상이면 multiclass로 취급되서 average 속성을 생략하면 에러가 발생된다.
# 클래스가 3개 이상이라면 None, 'micro', 'macro', 'weighted'중의 1개를 average 속성값으로 지정해야 한다.
# None: 각 클래스별 정밀도를 계산한다. => 이거 사용
# micro: 각 클래스에 대해 TP, FP, FN을 합한 뒤 재현율을 계산한다.
# macro: 각 클래스에 대해 재현율을 계산한뒤 산술 평균을 계산해서 재현율을 계산한다.
# weighted: 각 클래스에 대해 재현율을 계산한뒤 클래스별 데이터 비율에 따른 가중 평균으로 재현율을 계산한다.
precision = precision_score(y_test, predict, average=None)
print(precision)

[1.         0.9375     0.88888889]


In [6]:
# 재현율 평가
# recall_score() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 정밀도를 계산한다.
# average 속성으로 재현율 계산 방법을 지정한다. 정밀도와 사용 방법이 같다.
recell = recall_score(y_test, predict, average=None)
print(recell)

[1.         0.9375     0.88888889]


In [7]:
# f1 score 평가
# f1_score() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 f1 score를 계산한다.
# average 속성으로 f1 score 계산 방법을 지정한다. 정밀도, 재현율과 사용 방법이 같다.
f1 = f1_score(y_test, predict, average=None)
print(f1)

[1.         0.9375     0.88888889]


In [8]:
# 혼동 행렬
# confusion_matrix() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 혼동 행렬를 만든다.
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  1  8]]


In [9]:
# 분류 리포트
# classification_report() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 분류 리포트를 만든다.
# target_names 속성으로 분류 리포트에 레이블의 실제값을 출력할 수 있다.
class_report = classification_report(y_test, predict, target_names=raw_data.target_names) 
print(class_report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.94      0.94      0.94        16
   virginica       0.89      0.89      0.89         9

    accuracy                           0.95        38
   macro avg       0.94      0.94      0.94        38
weighted avg       0.95      0.95      0.95        38



모델 테스트

In [10]:
newData = [
    [50.1, 3.5, 1.4, 0.2], [4.9, 3. , 1.4, 0.2], [4.7, 3.2, 1.3, 0.2], [4.6, 3.1, 1.5, 0.2], [5. , 3.6, 1.4, 0.2],
    [5.4, 3.9, 1.7, 0.4], [4.6, 3.4, 1.4, 0.3], [5. , 3.4, 1.5, 0.2], [4.4, 2.9, 1.4, 0.2], [4.9, 3.1, 1.5, 0.1],
    [5.4, 3.7, 1.5, 0.2], [4.8, 3.4, 1.6, 0.2], [4.8, 3. , 1.4, 0.1], [4.3, 3. , 1.1, 0.1], [5.8, 4. , 1.2, 0.2],
    [5.7, 4.4, 1.5, 0.4], [5.4, 3.9, 1.3, 0.4], [5.1, 3.5, 1.4, 0.3], [5.7, 3.8, 1.7, 0.3], [5.1, 3.8, 1.5, 0.3]
]
x_newData = []
y_newData = [[0] for _ in range(20)]

for i in range(len(newData)):
    x_newData.append(std_scale.transform([newData[i]]))
    print(newData[i], x_newData[i], y_newData[i])

[50.1, 3.5, 1.4, 0.2] [[50.97140298  1.03217045 -1.33681519 -1.31530348]] [0]
[4.9, 3.0, 1.4, 0.2] [[-1.13740989 -0.11925475 -1.33681519 -1.31530348]] [0]
[4.7, 3.2, 1.3, 0.2] [[-1.36797986  0.34131533 -1.39259884 -1.31530348]] [0]
[4.6, 3.1, 1.5, 0.2] [[-1.48326484  0.11103029 -1.28103155 -1.31530348]] [0]
[5.0, 3.6, 1.4, 0.2] [[-1.0221249   1.26245549 -1.33681519 -1.31530348]] [0]
[5.4, 3.9, 1.7, 0.4] [[-0.56098497  1.95331061 -1.16946426 -1.058211  ]] [0]
[4.6, 3.4, 1.4, 0.3] [[-1.48326484  0.80188541 -1.33681519 -1.18675724]] [0]
[5.0, 3.4, 1.5, 0.2] [[-1.0221249   0.80188541 -1.28103155 -1.31530348]] [0]
[4.4, 2.9, 1.4, 0.2] [[-1.71383481 -0.34953979 -1.33681519 -1.31530348]] [0]
[4.9, 3.1, 1.5, 0.1] [[-1.13740989  0.11103029 -1.28103155 -1.44384972]] [0]
[5.4, 3.7, 1.5, 0.2] [[-0.56098497  1.49274053 -1.28103155 -1.31530348]] [0]
[4.8, 3.4, 1.6, 0.2] [[-1.25269487  0.80188541 -1.2252479  -1.31530348]] [0]
[4.8, 3.0, 1.4, 0.1] [[-1.25269487 -0.11925475 -1.33681519 -1.44384972]] [0

In [11]:
for i in range(len(newData)):
    predict = clf.predict(x_newData[i])
    print(y_newData[i], predict)

[0] [2]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
